In [3]:
%pip install scipy

  Using cached scipy-1.15.0-cp310-cp310-macosx_14_0_arm64.whl.metadata (61 kB)
Using cached scipy-1.15.0-cp310-cp310-macosx_14_0_arm64.whl (24.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import kagglehub


path = kagglehub.dataset_download("jay7080dev/rice-plant-diseases-dataset")
print("Path to dataset files:", path)


base_dir = os.path.join(path, "rice leaf diseases dataset")  


datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()} 
print("Class indices:", class_indices)
print("Class labels:", class_labels)


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)


loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

model.save("rice_disease_model.h5")
np.save("class_labels.npy", class_labels)  
print("Model and labels saved successfully.")

/Users/macbook/miniconda3/envs/rice/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/macbook/.cache/kagglehub/datasets/jay7080dev/rice-plant-diseases-dataset/versions/1
Found 3748 images belonging to 3 classes.
Found 936 images belonging to 3 classes.
Class indices: {'Bacterialblight': 0, 'Brownspot': 1, 'Leafsmut': 2}
Class labels: {0: 'Bacterialblight', 1: 'Brownspot', 2: 'Leafsmut'}


/Users/macbook/miniconda3/envs/rice/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/macbook/miniconda3/envs/rice/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.5672 - loss: 0.8924

/Users/macbook/miniconda3/envs/rice/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


117/117 ━━━━━━━━━━━━━━━━━━━━ 33s 276ms/step - accuracy: 0.5680 - loss: 0.8913 - val_accuracy: 0.8028 - val_loss: 0.5755
Epoch 2/30
  1/117 ━━━━━━━━━━━━━━━━━━━━ 31s 270ms/step - accuracy: 0.7188 - loss: 0.7239

/Users/macbook/miniconda3/envs/rice/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


117/117 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.7188 - loss: 0.7239 - val_accuracy: 0.7823 - val_loss: 0.5905
Epoch 3/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 34s 290ms/step - accuracy: 0.7766 - loss: 0.5949 - val_accuracy: 0.7856 - val_loss: 0.5501
Epoch 4/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.8125 - loss: 0.4828 - val_accuracy: 0.7716 - val_loss: 0.5559
Epoch 5/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 39s 336ms/step - accuracy: 0.8302 - loss: 0.4470 - val_accuracy: 0.8491 - val_loss: 0.3888
Epoch 6/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8125 - loss: 0.3392 - val_accuracy: 0.8642 - val_loss: 0.3621
Epoch 7/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 38s 324ms/step - accuracy: 0.8470 - loss: 0.3915 - val_accuracy: 0.8599 - val_loss: 0.3585
Epoch 8/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7500 - loss: 0.7489 - val_accuracy: 0.8685 - val_loss: 0.3240
Epoch 9/30
117/117 ━━━━━━━━━━━━━━━━━━━━ 40s 341ms/step - accuracy: 0.8683 - loss: 0.3502 - val_accurac

Validation Loss: 0.17845699191093445
Validation Accuracy: 0.9316239356994629
Model and labels saved successfully.
